In [ ]:
####trW
if (!require(MASS)) {
  install.packages("MASS")
  library(MASS)
}

tr <- function(matrix){
    sum(diag(matrix))
    }
simulateVC_old <- function(n, m, N, s2_gxg, s2_e){
#Generates X, gamma, y
 times <- list()
 t0 <- proc.time()
 X = matrix(rnorm(n*m),nrow = n,ncol =m)
 XT = t(X)
 p = m*(m-1)/2
 p_hat = m^2
 D = (X^2) %*% t(X^2)
 K = X %*% t(X)
 Q = K^2%*%D
 W = (K*K - D)/(2*p)
 trW = tr(W)
 trW2 =(tr(K^2 %*% K^2) - 2*tr(Q) + tr(D%*%D))/(4*p^2)
 times$exact <- (proc.time() - t0)["elapsed"]   
  #gamma = rnorm(p, mean = 0, sd = sqrt(s2_gxg / p))
  #y <- mvrnorm(mu = as.vector(H %*% gamma), Sigma = s2_e * diag(n))
  t1 <- proc.time()
  y <- mvrnorm(mu = rep(0, n), Sigma = (1/(2*p))*(K*K - D)*s2_gxg  + s2_e * diag(n))
  Dy <- diag(c(y))
  times$y <- (proc.time() - t1)["elapsed"]
  #### appromation part
  #### trW^2
  t2 <- proc.time()
  row_sq <- rowSums(X^2)
  Zmat <- replicate(N, {
    u1 <- rnorm(m)
    v  <- X %*% u1
    v^2 - row_sq
  })
  z_hat <- rowSums(Zmat) / (2 * sqrt(N))

     Dz <- diag(c(z_hat))
     element1<-sum(replicate(N, {
       u2 <- rnorm(n) 
       val<-t(u2)%*%X%*%XT%*%Dz%*%X%*%XT%*%Dz%*%u2
     }))/(2*p^2*N)#
     est_trW2 = c(element1 - (t(z_hat)%*%(X*X)%*%t((X*X))%*%z_hat)/(2*p^2))
    times$trW2 <- (proc.time() - t2)["elapsed"]
   #### trW checked  
    t3 <- proc.time()
   res_mat <- replicate(N, {
      u <- rnorm(m)           
      v <- X %*% u           
      (v^2 - rowSums(X^2))^2     
    })
    est_trW <- sum(res_mat) / (4*p*N)
    times$trW <- (proc.time() - t3)["elapsed"]
    #### ytWy  
    t4 <- proc.time()
     est_kk<- sum(replicate(N, {
       u <- rnorm(n)
       value <- (t(u) %*% Dy %*% K %*% Dy %*% K %*% u)
     }))/N
    
    est_yTW_haty <- est_kk/p_hat
    est_yTWy<- (p_hat*est_yTW_haty - t(y)%*% ((X^2) %*% t(X^2))%*%y)/(2*p)
    times$ytWy <- (proc.time() - t4)["elapsed"]
     ####the \theat_hat 
    t5 <- proc.time()
     s2_gxg_hat = (n* (y) %*% W %*%y - trW*(y) %*%y)/(n*trW2 - trW^2)
     s2_e_hat = ((y) %*% W %*%y*trW-(y) %*%y*trW2)/(trW^2-n*trW2)

     s2_gxg_app = (n* est_yTWy - est_trW*(y) %*%y)/(n*est_trW2 - est_trW^2)
     s2_e_app = (est_yTWy * est_trW-(y) %*%y*est_trW2)/(est_trW^2-n*est_trW2)
     times$solve <- (proc.time() - t5)["elapsed"]
    result <- c(s2_gxg_hat, s2_e_hat, s2_gxg_app, s2_e_app,
              trW, est_trW, trW2, est_trW2,
              (y %*% W %*% y), est_yTWy)
    list(timing = times, output = result)
}

In [ ]:
simulateVC_old(100, 100, 100, 0.7, 0.3)

In [127]:
simulateVC_old(10000, 1000, 100, 0.7, 0.3)

$timing
$timing$exact
elapsed 
  35.72 

$timing$y
elapsed 
 105.41 

$timing$trW2
elapsed 
  37.48 

$timing$trW
elapsed 
    7.3 

$timing$ytWy
elapsed 
  59.44 

$timing$solve
elapsed 
   0.34 


$output
 [1]     0.8852273     0.1320719    -0.2044371     1.2232878 10000.4217053
 [6] 10074.0586374 10281.5541117 10717.3237822 10422.2877176 10132.4536790

In [138]:
simulateVC_new(10000, 1000, 100, 0.7, 0.3)

$timing
$timing$exact
elapsed 
  34.85 

$timing$y
elapsed 
  106.6 

$timing$trW2
elapsed 
  18.31 

$timing$trW
elapsed 
   1.63 

$timing$ytWy
elapsed 
  17.78 

$timing$solve
elapsed 
   0.15 


$output
 [1] 7.006948e-01 3.091557e-01 5.102790e-01 4.997477e-01 9.999416e+03
 [6] 9.995745e+03 1.028017e+04 1.037246e+04 1.029464e+04 1.028820e+04

In [137]:
####trW
simulateVC_new <- function(n, m, N, s2_gxg, s2_e){
#Generates X, gamma, y
 times <- list()
 t0 <- proc.time()
 X = matrix(rnorm(n*m),nrow = n,ncol =m)
 XT = t(X)
 p = m*(m-1)/2
 p_hat = m^2
 D = (X^2) %*% t(X^2)
 K = X %*% t(X)
 Q = K^2%*%D
 W = (K*K - D)/(2*p)
 trW = tr(W)
 rowsumX2 = rowSums(X^2)
 trW2 =(tr(K^2 %*% K^2) - 2*tr(Q) + tr(D%*%D))/(4*p^2)
 times$exact <- (proc.time() - t0)["elapsed"]   
    
  #gamma = rnorm(p, mean = 0, sd = sqrt(s2_gxg / p))
  #y <- mvrnorm(mu = as.vector(H %*% gamma), Sigma = s2_e * diag(n))
  t1 <- proc.time()
  y <- mvrnorm(mu = rep(0, n), Sigma = (1/(2*p))*(K*K - D)*s2_gxg  + s2_e * diag(n))
  Dy <- diag(c(y))
  times$y <- (proc.time() - t1)["elapsed"]

    
  #### appromation part
  #### trW^2
  t2 <- proc.time()
  Zmat <- replicate(N, {
    u1 <- rnorm(m)
    v  <- X %*% u1
    v^2 - rowsumX2
  })
    z_hat <- rowSums(Zmat) / (2 * sqrt(N))
     Dz <- diag(c(z_hat))
    ##original method
     #t(u2)%*%X%*%XT%*%Dz%*%X%*%XT%*%Dz%*%u2
    
    #method 1 
    ## ensure every thing is matirx-vector
    element1 <- sum(replicate(N, {u2 <- rnorm(n); 
                               v1 <- X %*% (t(X) %*% (z_hat * u2)); 
                               v2 <- z_hat * (X %*% (t(X) %*% u2));
                               t(v2) %*% v1 }))/(2*p^2*N)
    #method 2 
    #KDz <- K %*% Dz
    #KDz2 <- KDz%*%KDz
    #element1<-sum(replicate(N, {
    #   u2 <- rnorm(n) 
    #   val<-t(u2)%*%KDz2%*%u2
    # }))/(2*p^2*N)

    est_trW2 = c(element1 - (t(z_hat)%*%(X*X)%*%t((X*X))%*%z_hat)/(2*p^2))
    times$trW2 <- (proc.time() - t2)["elapsed"]


    
   #### trW checked    
    t3 <- proc.time()
   res_mat <- replicate(N, {
      u <- rnorm(m)           
      v <- X %*% u           
      (v^2 - rowsumX2)^2     
    })
    est_trW <- sum(res_mat) / (4*p*N)
    times$trW <- (proc.time() - t3)["elapsed"]

    
    #### ytWy  
    t4 <- proc.time()
    #method 1 
    ## ensure every thing is matirx-vector
    est_kk <- sum(replicate(N, {u3 <- rnorm(n); 
                             v1 <- X %*% (t(X) %*% (y * u3)); 
                             v2 <- y * (X %*% (t(X) %*% u3));
                             t(v2) %*% v1 }))/N
    
   #DyK = Dy %*% K
   #Dyk2 = DyK %*% DyK
   #est_kk<- sum(replicate(N, {
   #   u3 <- rnorm(n)
   #   value <- (t(u3) %*% Dyk2 %*% u3)
   # }))/N

    # value <- (t(u) %*% Dy %*% K %*% Dy %*% K %*% u)
    est_yTW_haty <- est_kk/p_hat
    est_yTWy<- (p_hat*est_yTW_haty - t(y)%*% ((X^2) %*% t(X^2))%*%y)/(2*p)
    times$ytWy <- (proc.time() - t4)["elapsed"]
     ####the \theat_hat 
     t5 <- proc.time()
     exact_LHS <- matrix(c(trW2, trW, trW, n), 2, 2)
     exact_RHS <- c(t(y) %*%W %*% y, t(y)%*% y)
     var_exact <- solve(exact_LHS, exact_RHS) 

     app_LHS <- matrix(c(est_trW2, est_trW, est_trW, n), 2, 2)
     app_RHS <- c(est_yTWy, t(y)%*% y)
     var_app <- solve(app_LHS, app_RHS) 
     times$solve <- (proc.time() - t5)["elapsed"]
    result <- c(var_exact[1], var_exact[2], var_app[1], var_app[2],
              trW, est_trW, trW2, est_trW2,
              (y %*% W %*% y), est_yTWy)
    list(timing = times, output = result)
}

In [136]:
n = 5
m = 3
N = 5
s2_gxg = 0.7
s2_e = 0.3
p = m*(m-1)/2
X = matrix(rnorm(n*m),nrow = n,ncol =m)
K = X %*% t(X)
D = (X^2) %*% t(X^2)
y <- mvrnorm(mu = rep(0, n), Sigma = (1/(2*p))*(K*K - D)*s2_gxg  + s2_e * diag(n))
Dy <- diag(c(y))
u <- rnorm(n)
DyK= Dy %*% K
rowsumX2 = rowSums(X^2)
Zmat <- replicate(N, {
    u1 <- rnorm(m)
    v  <- X %*% u1
    v^2 - rowsumX2
})
z_hat <- rowSums(Zmat) / (2 * sqrt(N))


Dz <- diag(c(z_hat))
z_hat


[1] -7.672950 -2.686478 -1.178963 -1.185158 -5.553718

In [23]:
res <-as.data.frame(simulateVC(400, 100, 100, 0.7, 0.3)$output,colname = F)
write.table(res,"sdasdasd", row.names=FALSE, col.names=FALSE, quote=FALSE) 

r<-read.table("sdasdasd")
w<-read.table("sdasdasd")
cbind(r,w)

V1,V1
<dbl>,<dbl>
0.5553190,0.5553190
0.4545240,0.4545240
-0.1887499,-0.1887499
1.2068930,1.2068930
398.1902776,398.1902776
422.9138866,422.9138866
458.0650298,458.0650298
417.5661578,417.5661578
435.3592714,435.3592714


In [39]:
result <- vector("list", 100)
for (i in 1:100) {
    result[[i]] <- simulateVC(1000, 1000, 200, 0.7, 0.3)$output
}
data <- as.data.frame(do.call(cbind, lapply(result, unlist)))
write.csv(data, "fixm_1000_1000_rnorm")

[1]    0.5812468    0.4057482    0.5701238    0.4137134 1993.0122610
 [6] 2003.9536987 2048.6574634 2060.6111333 1999.4367049 2003.8658219

In [5]:
result <- vector("list", 25)
for (i in 1:25) {
    result[[i]] <- simulateVC(10000, 2500, 100, 0.7, 0.3)$output
}

data <- as.data.frame(do.call(cbind, lapply(result, unlist)))
write.csv(data, "10k_1")

In [6]:
result <- vector("list", 25)
for (i in 1:25) {
    result[[i]] <- simulateVC(10000, 2500, 100, 0.7, 0.3)$output
}

data <- as.data.frame(do.call(cbind, lapply(result, unlist)))
write.csv(data, "10k_2")

In [7]:
result <- vector("list", 25)
for (i in 1:25) {
    result[[i]] <- simulateVC(10000, 2500, 100, 0.7, 0.3)$output
}

data <- as.data.frame(do.call(cbind, lapply(result, unlist)))
write.csv(data, "10k_3")

In [8]:
result <- vector("list", 25)
for (i in 1:25) {
    result[[i]] <- simulateVC(10000, 3750, 100, 0.7, 0.3)$output
}

data <- as.data.frame(do.call(cbind, lapply(result, unlist)))
write.csv(data, "10k_4")

In [32]:
simulateVC_timed(2000, 500, 100, 0.7, 0.3)

$timing
$timing$block1
elapsed 
    0.5 

$timing$block2
elapsed 
   2.05 

$timing$block3
elapsed 
  25.16 

$timing$block4
elapsed 
  14.54 

$timing$block5
elapsed 
   3.18 

$timing$block6
elapsed 
   0.01 


$output
 [1]    0.8340818    0.1953192  -21.0501779   21.7852399 2006.8409190
 [6] 1971.7634692 2078.6031011 1941.3961184 2125.6997124 2088.6065567

In [30]:
n = 4000
m = 1000
s2_gxg = 0.7
s2_e = 0.3
N = 100
p = m*(m-1)/2
X     <- matrix(rnorm(n*m), nrow = n,    ncol = m)
XT <- t(X)
t0 <- proc.time()
   z_hat = matrix(0, nrow = n, ncol = 1)
     result<- replicate(N, {
       for (i in 1:n) {
         u1 <-rnorm(m)
         z_hat[i,] <-t(u1)%*%X[i,]%*%X[i,]%*%u1-t(X[i,])%*%X[i,]
       }
       z_hat
     })
     z_hat <- apply(result, 1, sum)/(2*sqrt(N))
     Dz <- diag(c(z_hat))
     element1<-sum(replicate(N, {
       u2 <- rnorm(n) 
       val<-t(u2)%*%X%*%XT%*%Dz%*%X%*%XT%*%Dz%*%u2
     }))/(2*p^2*N)
     est_trW2 = c(element1 - (t(z_hat)%*%(X*X)%*%t((X*X))%*%z_hat)/(2*p^2))
proc.time()-t0
print(est_trW2)
t1 <- proc.time()
 row_sq <- rowSums(X^2)
  Zmat <- replicate(N, {
    u1 <- rnorm(m)
    v  <- X %*% u1
    v^2 - row_sq
  })
  z_hat <- rowSums(Zmat) / (2 * sqrt(N))

     Dz <- diag(c(z_hat))
     element1<-sum(replicate(N, {
       u2 <- rnorm(n) 
       val<-t(u2)%*%X%*%XT%*%Dz%*%X%*%XT%*%Dz%*%u2
     }))/(2*p^2*N)
     est_trW2 = c(element1 - (t(z_hat)%*%(X*X)%*%t((X*X))%*%z_hat)/(2*p^2))
proc.time()-t1
print(est_trW2)

   user  system elapsed 
  79.55    4.00   78.48 

[1] 4037.449


   user  system elapsed 
  15.25    0.43    8.95 

[1] 4084.555


In [60]:
res_mat <- replicate(N, {
  u <- rnorm(m)           # draw m-vector
  v <- X %*% u            # n-vector: each entry = X[t,] %*% u
  (v^2 - rowSums(X^2))^2        # vector of length n
})

trW_hat <- sum(res_mat) / (4 * p * N)
trW_hat

[1] 2.117612

In [61]:
est_trW <- sum(replicate(N, {
    u1 <- rnorm(m)
    sum(sapply(1:n, function(i) {
      ((t(u1)%*%X[i,]%*%X[i,]%*%u1) - (t(X[i,])%*%X[i,]))^2
    }))
  })) / (4*p*N)
est_trW

[1] 2.035443